In [92]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import sys
from PyQt5.QtWidgets import *
from PyQt5.QtCore import *
from PyQt5.QtGui import *
from skimage.draw import polygon

import PyQt5.QtGui as QtGui
import PyQt5.QtCore as QtCore

import cv2
from cv2 import imwrite as imwrite


import numpy as np
import shapefile as sf
import sklearn.decomposition as sd





inputfile = sf.Reader("/g/data1a/ge3/AEM_Model/3D_AEM_model_V3.shp")
inputrecord = np.array(inputfile.records())

xyz = inputrecord[:,10:13].astype(float) #nx3 matrix

height = len(np.unique(xyz[:,2], axis=0))
width = len(np.unique(xyz[:,0:2], axis=0))

#project x, y onto a line
pca = sd.PCA(n_components=1)
pca.fit(xyz[:,0:2])
xy = pca.transform(xyz[:,0:2])

print(inputrecord.shape)
print(xy.shape)

width=800
height=600

point = np.hstack([inputrecord,xy.reshape(-1,1),inputrecord[:,12].reshape(-1,1)])



img = np.zeros([height,width,3],dtype=float)

bucket = [[[] for h in range(height)] for w in range(width)]

point[:,-1] = np.round(
    (point[:,-1].astype(float) - point[:,-1].astype(float).min())
    /(point[:,-1].astype(float).max() - point[:,-1].astype(float).min())
    *(height-1)
   )
point[:,-2] = np.round(
    (point[:,-2].astype(float) - point[:,-2].astype(float).min())
    /(point[:, -2].astype(float).max() - point[:, -2].astype(float).min())
    *(width-1)
   )
bucket_size = np.zeros([height,width,3],dtype=np.int)

for p in point:
    if p[1]!='Surface':
        p = p[[2,3,4,-2,-1]].astype(float)
        p[-1]=height-1-p[-1]
        bucket[int(p[-2])][int(p[-1])].append(p)
        img[int(p[-1]), int(p[-2]), 0] = img[int(p[-1]), int(p[-2]), 0]+p[0]
        img[int(p[-1]), int(p[-2]), 1] = img[int(p[-1]), int(p[-2]), 1]+p[1]
        img[int(p[-1]), int(p[-2]), 2] = img[int(p[-1]), int(p[-2]), 2]+p[2]
        bucket_size[int(p[-1]), int(p[-2]), :] = bucket_size[int(p[-1]), int(p[-2]), :]+1

count = 0

for i in range(bucket_size.shape[0]):
    for j in range(bucket_size.shape[1]):
        for k in range(bucket_size.shape[2]):
            if bucket_size[i,j,k]==0:
                bucket_size[i, j, k]=1
            else:
                count = count+bucket_size[i,j,k]
print(count)

img = img/bucket_size

minr = img[:,:,0][np.nonzero(img[:,:,0])].min()
ming = img[:,:,1][np.nonzero(img[:,:,0])].min()
minb = img[:,:,2][np.nonzero(img[:,:,0])].min()

maxr = img[:,:,0].max()
maxg = img[:,:,1].max()
maxb = img[:,:,2].max()

img[:,:,0][np.nonzero(img[:,:,0])] = img[:,:,0][np.nonzero(img[:,:,0])]+(maxr-minr)*0.5
img[:,:,1][np.nonzero(img[:,:,1])] = img[:,:,1][np.nonzero(img[:,:,1])]+(maxg-ming)*0.5
img[:,:,2][np.nonzero(img[:,:,2])] = img[:,:,2][np.nonzero(img[:,:,2])]+(maxb-minb)*0.5

print(minr,ming,minb)
print(maxr,maxg,maxb)

img[:, :, 0] = (img[:, :, 0]) * 255 / (maxr + (maxr - minr)*0.5)
img[:, :, 1] = (img[:, :, 1]) * 255 / (maxg + (maxg - ming)*0.5)
img[:, :, 2] = (img[:, :, 2]) * 255 / (maxb + (maxb - minb)*0.5)

KeyboardInterrupt: 

In [91]:
cv2.imwrite('gui.png',img.astype(np.uint8))

True

In [89]:
x=np.array([[1,0],[2,0]])
x[np.nonzero(x)]

array([1, 2])

In [66]:

bucket_size = np.zeros([height,width,3],dtype=np.int)

for p in point:
    if p[1]!='Surface':
        p = p[[2,3,4,-2,-1]].astype(float)

        bucket[int(p[-2])][int(p[-1])].append(p)
        img[int(p[-1]), int(p[-2]), 0] = img[int(p[-1]), int(p[-2]), 0]+p[0]
        img[int(p[-1]), int(p[-2]), 1] = img[int(p[-1]), int(p[-2]), 1]+p[1]
        img[int(p[-1]), int(p[-2]), 2] = img[int(p[-1]), int(p[-2]), 2]+p[2]
        bucket_size[int(p[-1]), int(p[-2]), :] = bucket_size[int(p[-1]), int(p[-2]), :]+1

for i in range(bucket_size.shape[0]):
    for j in range(bucket_size.shape[1]):
        for k in range(bucket_size.shape[2]):
            if bucket_size[i,j,k]==0:
                bucket_size[i, j, k]=1

img = img/bucket_size

minr = img[:,:,0].min()
ming = img[:,:,1].min()
minb = img[:,:,2].min()

maxr = img[:,:,0].max()
maxg = img[:,:,1].max()
maxb = img[:,:,2].max()

print(minr,ming,minb)
print(maxr,maxg,maxb)

img[:, :, 0] = (img[:, :, 0] - minr) * 255 / (maxr - minr)
img[:, :, 1] = (img[:, :, 1] - ming) * 255 / (maxg - ming)
img[:, :, 2] = (img[:, :, 2] - minb) * 255 / (maxb - minb)


0.0 0.0 0.0
256.83 256.03000000000003 256.79


In [67]:
img

array([[[0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [1.31966004e-04, 5.38446028e-04, 2.52567197e-04],
        [8.73647507e-03, 1.58170919e-01, 2.12962350e-02],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],

       [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [4.20933283e-04, 4.39289015e-03, 9.29497637e-04],
        [3.06972911e-04, 4.52020388e-03, 7.85769916e-04],
        [1.16978241e-03, 1.86413807e-02, 2.81182497e-03]],

       [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [4.05801432e-04, 7.00

In [58]:

class myshapefile:
    def __init__(self):
        inputfile = sf.Reader("/g/data1a/ge3/AEM_Model/3D_AEM_model_V3.shp")
        inputrecord = np.array(inputfile.records())

        xyz = inputrecord[:,10:13].astype(float) #nx3 matrix

        height = len(np.unique(xyz[:,2], axis=0))
        width = len(np.unique(xyz[:,0:2], axis=0))

        #project x, y onto a line
        pca = sd.PCA(n_components=1)
        pca.fit(xyz[:,0:2])
        xy = pca.transform(xyz[:,0:2])

        print(inputrecord.shape)
        print(xy.shape)


        self.point = np.hstack([inputrecord,xy.reshape(-1,1),inputrecord[:,12].reshape(-1,1)])





    def getimage(self,width,height):
        img = np.zeros([height,width,3],dtype=float)

        self.bucket = [[[] for h in range(height)] for w in range(width)]

        self.point[:,-1] = np.round(
                                    (self.point[:,-1].astype(float) - self.point[:,-1].astype(float).min())
                                    /(self.point[:,-1].astype(float).max() - self.point[:,-1].astype(float).min())
                                    *(height-1)
                                   )
        self.point[:,-2] = np.round(
                                    (self.point[:,-2].astype(float) - self.point[:,-2].astype(float).min())
                                    /(self.point[:, -2].astype(float).max() - self.point[:, -2].astype(float).min())
                                    *(width-1)
                                   )

        bucket_size = np.zeros([height,width,3],dtype=np.int)

        for p in self.point:
            if p[1]!='Surface':
                p = p[[2,3,4,-2,-1]].astype(float)

                p[-1]=height-p[-1]-1

                self.bucket[int(p[-2])][int(p[-1])].append(p)
                img[int(p[-1]), int(p[-2]), 0] = img[int(p[-1]), int(p[-2]), 0]+p[0]
                img[int(p[-1]), int(p[-2]), 1] = img[int(p[-1]), int(p[-2]), 1]+p[1]
                img[int(p[-1]), int(p[-2]), 2] = img[int(p[-1]), int(p[-2]), 2]+p[2]
                bucket_size[int(p[-1]), int(p[-2]), :] = bucket_size[int(p[-1]), int(p[-2]), :]+1

        for i in range(bucket_size.shape[0]):
            for j in range(bucket_size.shape[1]):
                for k in range(bucket_size.shape[2]):
                    if bucket_size[i,j,k]==0:
                        bucket_size[i, j, k]=1

        img = img/bucket_size



        #return(np.stack([imgr,imgg,imgb],axis=2))
        return(img)





In [59]:

x=myshapefile()
y=x.getimage(800,600)
cv2.imwrite('gui.png',y)

(1309936, 15)
(1309936, 1)


True

In [62]:
cv2.imwrite('gui.png',y)

True

In [46]:
img[:, :, 0] = (img[:, :, 0] - minr) * 255 / (maxr - minr)
img[:, :, 1] = (img[:, :, 1] - ming) * 255 / (maxg - ming)
img[:, :, 2] = (img[:, :, 2] - minb) * 255 / (maxb - minb)

array([[[0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        ...,
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ]],

       [[0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        ...,
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ]],

       [[0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        ...,
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ]],

       ...,

       [[0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0

In [22]:
x

array([ 1, 18,  3])